In [76]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ai-201-b-mse-2-ai-d/sample_submission.csv
/kaggle/input/ai-201-b-mse-2-ai-d/train.csv
/kaggle/input/ai-201-b-mse-2-ai-d/test.csv


In [77]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier

In [78]:
train = pd.read_csv("/kaggle/input/ai-201-b-mse-2-ai-d/train.csv")
test = pd.read_csv("/kaggle/input/ai-201-b-mse-2-ai-d/test.csv")


In [79]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18306 entries, 0 to 18305
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Gender                          17071 non-null  object 
 1   Age                             16495 non-null  float64
 2   Height                          16702 non-null  float64
 3   Weight                          16838 non-null  float64
 4   family_history_with_overweight  17241 non-null  object 
 5   FAVC                            17257 non-null  object 
 6   FCVC                            17331 non-null  float64
 7   NCP                             16586 non-null  float64
 8   CAEC                            16841 non-null  object 
 9   SMOKE                           16759 non-null  object 
 10  CH2O                            17373 non-null  float64
 11  SCC                             16503 non-null  object 
 12  FAF                             

In [80]:
train.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Male,33.226808,1.766888,83.337721,yes,NaN,2.0,NaN,Sometimes,no,1.964435,no,0.750111,0.0,no,Automobile,Overweight_Level_II
1,Female,18.000000,1.600000,55.000000,yes,yes,2.0,3.000000,Sometimes,no,2.000000,no,1.000000,1.0,Sometimes,Public_Transportation,Normal_Weight
2,NaN,26.000000,1.640741,111.930700,yes,yes,3.0,3.000000,Sometimes,no,2.617988,no,0.000000,NaN,Sometimes,Public_Transportation,Obesity_Type_III
3,Male,19.000000,1.690000,64.000000,no,yes,2.0,NaN,Frequently,no,2.000000,no,0.000000,1.0,Sometimes,Automobile,Normal_Weight
4,Male,NaN,1.606474,94.189167,yes,yes,2.0,2.812283,Sometimes,no,2.000000,no,1.000000,0.0,no,NaN,Obesity_Type_I


In [81]:
train.sample(5)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
3247,Male,21.000000,1.610000,100.000000,yes,yes,2.000000,1.000000,Sometimes,no,3.000000,no,1.000000,0.000000,Sometimes,Public_Transportation,Obesity_Type_I
12318,Male,24.982997,1.694969,99.982541,yes,yes,2.766612,2.427137,Sometimes,no,2.001208,no,1.089061,0.426465,no,Public_Transportation,Obesity_Type_II
8764,Female,20.254534,1.569480,NaN,no,yes,2.594653,1.000000,Frequently,no,1.000000,NaN,0.000000,0.138418,Sometimes,Public_Transportation,Insufficient_Weight
3675,Male,21.000000,NaN,NaN,no,yes,2.000000,3.000000,Frequently,no,2.000000,no,1.000000,2.000000,Sometimes,Walking,Overweight_Level_I
17214,Female,22.889099,1.633020,82.000000,yes,yes,1.000000,1.000000,Sometimes,no,1.636326,no,0.000000,0.069367,no,Public_Transportation,Obesity_Type_I


In [82]:
train.isnull().sum()

Gender                            1235
Age                               1811
Height                            1604
Weight                            1468
family_history_with_overweight    1065
FAVC                              1049
FCVC                               975
NCP                               1720
CAEC                              1465
SMOKE                             1547
CH2O                               933
SCC                               1803
FAF                               1724
TUE                               1120
CALC                              1090
MTRANS                            1089
NObeyesdad                           0
dtype: int64

In [83]:
test.isnull().sum()

id                                0
Gender                            0
Age                               0
Height                            0
Weight                            0
family_history_with_overweight    0
FAVC                              0
FCVC                              0
NCP                               0
CAEC                              0
SMOKE                             0
CH2O                              0
SCC                               0
FAF                               0
TUE                               0
CALC                              0
MTRANS                            0
dtype: int64

In [84]:
test_id = test['id']
test = test.drop(columns=['id'])

In [85]:
x=train.drop(columns=['NObeyesdad'])
y=train['NObeyesdad']

In [86]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [87]:
numeric_feature=x.select_dtypes(include=['int64','float64']).columns
categorical_features=x.select_dtypes(include=['object']).columns

In [88]:
numeric_pipeline=Pipeline(steps=[
    ('impute',SimpleImputer(strategy='mean')),
    ('scaler',StandardScaler())
])

In [89]:
categorical_pipeline=Pipeline(steps=[
    ('impute',SimpleImputer(strategy='most_frequent')),
    ('scaler',OneHotEncoder(handle_unknown='ignore'))
])

In [90]:
preprocessor=ColumnTransformer(transformers=[
    ('num',numeric_pipeline,numeric_feature),
    ('cat',categorical_pipeline,categorical_features)
])

In [91]:
model=GradientBoostingClassifier(
    n_estimators=920,        # number of trees
    learning_rate=0.025,      # smaller learning rate usually better
    max_depth=4,             # depth of each tree
    subsample=0.7,           # fraction of samples for each tree
    min_samples_split=5,     # minimum samples to split a node
    min_samples_leaf=2,      # minimum samples in a leaf
    max_features='sqrt',     # number of features to consider for best split
    random_state=42
)

In [92]:
pipeline=Pipeline(steps=[
    ('preprocessor',preprocessor),
    ('model',model)
])

In [93]:
pipeline.fit(x_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('scaler',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE',
       'SCC', 'CALC', 'MTRANS'],
      dtype='object'))])),
                ('model',
                 GradientBoostingClassifier(learning_rate=0.025, max_depth=4,
                                            max_features='sqrt',
                                            min_samples_leaf=2,
                                            min_samples_split=5,
                                            n_estimators=920, random_state=42,
                                            subsample=0.7))])

In [94]:
y_pred=pipeline.predict(x_test)

In [95]:
accu=accuracy_score(y_pred,y_test)

In [96]:
print(accu)

0.8806663025669034


In [97]:
y_final=pipeline.predict(test)

In [98]:
print(y)

0        Overweight_Level_II
1              Normal_Weight
2           Obesity_Type_III
3              Normal_Weight
4             Obesity_Type_I
                ...         
18301       Obesity_Type_III
18302        Obesity_Type_II
18303    Insufficient_Weight
18304    Insufficient_Weight
18305     Overweight_Level_I
Name: NObeyesdad, Length: 18306, dtype: object


In [99]:
print(y_final)

['Obesity_Type_III' 'Overweight_Level_II' 'Obesity_Type_I' ...
 'Obesity_Type_I' 'Overweight_Level_I' 'Overweight_Level_II']


In [100]:
submission = pd.DataFrame({'id':test_id,'NObeyesdad':y_final})

In [101]:
submission.to_csv('submission.csv', index=False)